In [1]:
import pandas as pd
import numpy as np

In [2]:
#Reading the train file
train_df= pd.read_csv("train.csv", parse_dates=['impression_time'] , index_col='impression_id')
# Parsing the dates and indexing the impression ids

In [3]:
final_train= train_df.drop(['app_code'], axis=1)

In [22]:
#final_train = final_train[(final_train['impression_time']>=pd.Timestamp(2018,11,15)) & 
                #            (final_train['impression_time']<pd.Timestamp(2018,12,12))]

In [4]:
#Encoding string columns
from sklearn.preprocessing import LabelEncoder

In [5]:
os_le = LabelEncoder()
final_train.loc[:,'os_label'] = os_le.fit_transform(final_train['os_version'])

In [6]:
final_input= final_train.drop(['impression_time','os_version'],axis='columns')
final_input.reset_index(inplace=True) # Removing impression id index as it might cause confusion
final_input.drop('impression_id', axis=1, inplace=True)
train_data = final_input.drop(['is_click'], axis=1)
target = final_input['is_click']

In [8]:
#Setting up a tree and training it
from sklearn.ensemble import RandomForestClassifier

In [9]:
model= RandomForestClassifier(criterion='entropy', n_estimators=20)
model.fit(train_data,target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
#Creating test dataframe
#Reading the train file
test_data= pd.read_csv("test.csv", parse_dates=['impression_time'])

In [11]:
test_data.drop(['impression_id', 'impression_time', 'app_code'], axis=1, inplace=True)

In [12]:
osle = LabelEncoder()
test_data.loc[:,'os_label'] = osle.fit_transform(test_data['os_version'])

In [13]:
test_data.drop('os_version', axis=1, inplace=True)

In [14]:
answer = model.predict(test_data)

In [15]:
#Creating submission file using dataframe
submit_data= pd.read_csv("test.csv")
submit_data.drop(['impression_time','user_id','app_code','os_version','is_4G'], axis=1, inplace=True)
submit_data.to_csv('submission.csv', index=False)

In [16]:
#Converting into np array
predict_array = np.array(answer)
np.shape(predict_array)

(90675,)

In [17]:
#Writing the predicted array to another file in a column
import csv
with open('prediction.csv', 'a', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for row in range(0,predict_array.shape[0]):
        myList = []
        myList.append(predict_array[row])
        writer.writerow(myList)

In [18]:
# Now bringing them together
a = pd.read_csv("submission.csv")
b = pd.read_csv("prediction.csv", header=None, names=["is_click"])
final_submission = pd.concat([a,b],axis=1)
final_submission.to_csv('final_submission.csv', index=False)

#The End